# Franka Robot Grasping Tutorial: With and Without Spinning Pads

This tutorial demonstrates a comparison between two Franka Emika Panda robots—one with spinning pads and one without—performing the same grasping task. The spinning pads reduce the torsional friction about the normal vector of the contact between robot's finger and the object. 
"The implementation for friction in genesis is not as advanced that in Mujoco. There is only 1 scalar parameter in Genesis, and condim cannot be changed (only condim = 3 is implemented). There is no torsional friction at the moment. The torsional friction in genesis is the result of multi-contact friction."


The standard robot uses joints 0-8, with 0-6 for arm movement and 7-8 for the gripper. The modified robot includes spinning pads at joints 9-10, which can help provide rotational slippage at the contact point

In [1]:
import genesis as gs
import numpy as np
import torch
from pytransform3d import (
    transformations as pt,
    rotations as pr,
    batch_rotations as pb,
    trajectories as ptr,
    plot_utils as ppu
)
from pandaSim.geometry.genesis_adapter import GenesisAdapter
from pandaSim.planning.screw_motion_planner import ScrewMotionPlanner


import spatialmath as sm
# auto reaload
%load_ext autoreload
%autoreload 2

[I 05/18/25 21:36:24.577 25073] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [2]:
gs.destroy()
gs.init(backend=gs.gpu)
gs.set_random_seed(seed=42)
adapter = GenesisAdapter()
planner = ScrewMotionPlanner()

[Genesis] [21:36:35] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [21:36:35] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [21:36:35] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [21:36:36] [INFO] Running on [NVIDIA GeForce RTX 3070 Laptop GPU] with backend gs.cuda. Device memory: 7.78 GB.
[Genesis] [21:36:36] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [21:36:36] [INFO] Scene <2ac7159> created.


## Scene Construction

Here we create the simulation environment with:
1. A ground plane
2. Two Franka Emika Panda robots:
   - One standard robot at position (0,2,0)
   - One with spinning pads and virtual finger at position (0,0,0)
3. Two cubes for each robot
4. A target entity for visualization

The key difference between the robots is that the standard Franka `(franka_MJCF)` uses only the arm and finger joints (joints 0-8), while the other version has additional spinning pads (joints 9-10) that cause rotational slippage using extra hinge DoFs.


In [12]:
scene = adapter.scene
plane = scene.add_entity(
    gs.morphs.Plane(),
)

franka = scene.add_entity(
    gs.morphs.MJCF(file="../assets/xml/franka_emika_panda/panda_vir.xml")
                   
)
cube_size = (0.1, 0.07, 0.25)
cube_pos = (0.7, 0, 0.05)

cube = scene.add_entity(
    gs.morphs.Box(
        size = cube_size,
        pos  = cube_pos,
        euler = (0, -90, 0)
    ),
    surface=gs.surfaces.Default(
        color=(0.5, 0.8, 0.94),
    ),
    material=gs.materials.Rigid(friction=5)
    
)

target_entity = scene.add_entity(
    gs.morphs.Mesh(
        file="meshes/axis.obj",
        scale=0.15,
        collision=False,
    ),
    surface=gs.surfaces.Default(color=(1, 0.5, 0.5, 1)),
    material=gs.materials.Rigid(gravity_compensation=1.0)
)

GenesisException: Scene is already built.

## Robot Configuration

We build the scene and define important joint groups:
- `motors_dof`: Joints 0-6 for arm movement
- `fingers_dof`: Joints 7-8 for gripper fingers
- `virtual_hand`: Joint 9 at the middle of the finger for rotational slippage (only on the `franka`)
- `spinning_pads`: Joints 9-10 for the spinning contact pads (only on the `franka`)

In [4]:
n_envs = 4
scene.build(n_envs=n_envs, env_spacing=(2, 2), compile_kernels=False)


[Genesis] [21:30:01] [INFO] Building scene <01ac936>...
[Genesis] [21:30:02] [WARNING] Reference robot position exceeds joint limits.
[Genesis] [21:30:05] [INFO] Building visualizer...
[Genesis] [21:30:07] [INFO] Viewer created. Resolution: 1280×960, max_FPS: 60.


In [5]:
for joint in franka.joints:
    print(joint.name, joint.idx_local)
motors_dof = np.arange(7)
fingers_dof = np.arange(7, 9)
virtual_finger = 9
spinning_pads = np.arange(10, 12)

joint1 0
joint2 1
joint3 2
joint4 3
joint5 4
joint6 5
joint7 6
finger_joint1 7
finger_joint2 8
virtual_finger_joint 9
left_pad_hinge 10
right_pad_hinge 11


## Initial Robot Positioning

Reset the scene and set both robots to the same initial joint configuration. We'll use this "ready pose" as a starting position for our task. Notice that the standard robot has 9 DOFs (degrees of freedom), while the modified version can have 1 dof extra for virtual_finger and two extras for spinning_pads

In [6]:
scene.reset()
qr = np.array([0.0000, -0.3002, 0.0000, -2.1991, 0.0000, 2.0071, 0.7854, 0.04, 0.04])
virtual_franka_qr = np.append(qr, [0, 0, 0])
ready_qpos = np.tile(virtual_franka_qr, (n_envs, 1))
# set joints position
franka.ready_qpos = ready_qpos
franka.set_qpos(franka.ready_qpos)
# position control
franka.control_dofs_position(franka.ready_qpos)


end_effector = franka.get_link('hand')
virtual_finger = franka.get_link('virtual_finger')



target_entity.set_pos(virtual_finger.get_pos())
target_entity.set_quat(virtual_finger.get_quat())


scene.step()

[Genesis] [21:30:07] [INFO] Resetting Scene <01ac936>.


In [11]:
scene.reset()
scene.step()
transform = adapter.transform(obj=cube, 
                  transformation=sm.SE3.RPY(np.pi/3, 0, 0).A, 
                  apply=True,
                  envs_idx=0)
scene.step()
obj_T_gripper = np.array([[0, 1, 0, 0],
                          [0, 0, -1, 0],
                          [-1, 0, 0, 0],
                          [0, 0, 0, 1]])

grasp_pose, qs, s_axes = planner.compute_grasp(obj=cube, 
                                          adapter=adapter, 
                                          prefer_closer_grasp=True,
                                          grasp_height='top',
                                          gripper_depth=0.03,
                                          gripper_offset=obj_T_gripper,
                                          output_type='pq')

target_entity.set_qpos(grasp_pose)
scene.step()

[Genesis] [21:35:25] [INFO] Resetting Scene <01ac936>.




[E 05/18/25 21:35:25.516 24680] [cuda_driver.h:operator()@92] CUDA Error CUDA_ERROR_ASSERT: device-side assert triggered while calling malloc_async_impl (cuMemAllocAsync)


RuntimeError: [cuda_driver.h:operator()@92] CUDA Error CUDA_ERROR_ASSERT: device-side assert triggered while calling malloc_async_impl (cuMemAllocAsync)

## Pre-grasp Positioning

Now we'll move both robots to a position above the cube. We'll use inverse kinematics to calculate the joint positions needed to place the end-effector at the desired position and orientation. 


In [10]:

qpos = franka.inverse_kinematics(
    link = virtual_finger,
    pos  = grasp_pose[..., :3],
    quat = grasp_pose[..., 3:]
)
qpos[fingers_dof] = 0.04


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# path = franka.plan_path(
#     qpos_goal     = qpos,
#     num_waypoints = 250, # 2s duration
# )

/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [0,0,0], thread: [0,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [0,0,0], thread: [1,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [0,0,0], thread: [2,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [0,0,0], thread: [3,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [0,0,0], thread: [4,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernel.cu:93: operator(): block: [0,0,0], thread: 

GenesisException: Motion planning is not supported for batched envs (yet).

## Path Planning

Generate a smooth trajectory for each robot to reach the pre-grasp position.

In [20]:
# execute the planned path
for waypoint in path:
    franka.control_dofs_position(waypoint)
    scene.step()

# allow robot to reach the last waypoint
for i in range(100):
    scene.step()


## Reaching for the Object

Calculate and execute a path to move from the pre-grasp position down to the actual grasp position. Both robots move their end-effectors to the side of their respective cubes.


In [21]:
franka.control_dofs_force(np.array([-2, -2]), fingers_dof)
adapter.step_simulation(1)

In [22]:
virtual_finger_pose = adapter.forward_kinematics(franka, virtual_finger)

cube_bbox = adapter.get_bbox(cube)
qs, s_axes = planner.screw_from_bbox(cube_bbox)
q, s_axis = qs[1], s_axes[1]


finger_pqs = planner.generate_screw_trajectory(initial_pose=virtual_finger_pose, q=q, s_axis=s_axis, output_type='pq')

qposs = []
for pq in finger_pqs:
    qpos = franka.inverse_kinematics(
        link = virtual_finger,
        pos = pq[:3],
        init_qpos = qpos        
    )
    qposs.append(qpos)

In [23]:

for waypoint in qposs:
    franka.control_dofs_position(waypoint[motors_dof], motors_dof)
    target_entity.set_pos(virtual_finger.get_pos())
    target_entity.set_quat(virtual_finger.get_quat())
    scene.step()



In [24]:
franka.control_dofs_force(np.array([0, 0]), fingers_dof)
adapter.step_simulation(1)